# Imports and Utilities

In [ ]:
import numpy as np
import bisect
import re
from collections import Counter, defaultdict
import itertools

In [ ]:
def read_input(day, map_f=lambda x:x.strip(), newline='\n'):
    with open(f"input{day}.txt", 'r') as f:
        return list(map(map_f, f.read().strip().split(newline)))

def binary_search(a, x, low=0, hi=None):
    if hi is None:
        hi = len(a)
    'Locate the leftmost value exactly equal to x'
    i = bisect.bisect_left(a, x, low, hi)
    if i != len(a) and a[i] == x:
        return i
    return -1

# Day 1

In [ ]:
target_sum = 2020
numbers_count = Counter(read_input(1, lambda x: int(x)))

def find_pair(numbers_count, target_sum):
    for n, count in numbers_count.items():
        left = target_sum - n
        if left == n and count >= 2:
            print(f"Numbers found: {n},{left}\tProduct: {left*n}")
            return
        elif left in numbers_count:
            print(f"Numbers found: {n},{left}\tProduct: {left*n}")
            return
    print(f"Pair with sum equal to {target_sum} not found.")


def find_triple(numbers_count, target_sum):
    for n1, count1 in numbers_count.items():
        for n2, count2 in numbers_count.items():
            if n1 == n2 and count1 <= 1:
                continue
            left = target_sum - n1 - n2
            if left < 0:
                continue
            if left == n1 == n2 and count1 >= 3:
                print(f"Numbers found: {n1},{n2},{left}\tProduct: {left*n1*n2}")
                return
            elif left in numbers_count:
                print(f"Numbers found: {n1},{n2},{left}\tProduct: {left*n1*n2}")
                return
    print(f"Triple with sum equal to {target_sum} not found.")

find_pair(numbers_count, target_sum)
find_triple(numbers_count, target_sum)

# Day 2

In [ ]:
password_list = read_input(2, lambda x: re.match(r"(\d+)\-(\d+) ([a-z]): ([a-z]+)", x).groups())

def count_valid_passwords(password_list):
    valid_m1 = 0
    valid_m2 = 0
    for l, h, c, password in password_list:
        l = int(l)
        h = int(h)
        counter = Counter(password)
        if l <= counter[c] <= h:
            valid_m1 += 1
            
        is_l = password[l-1] == c
        is_h = password[h-1] == c

        if is_l ^ is_h:
            valid_m2 += 1

    print(f"Valid passwords according to first method: {valid_m1}")
    print(f"Valid passwords according to second method: {valid_m2}")

count_valid_passwords(password_list)

# Day 3

In [ ]:
map_t = read_input(3)

def count_trees_on_slope(map_t, slope):
    d_j, d_i = slope
    rows = len(map_t)
    cols = len(map_t[0])
    i, j = 0, 0
    trees = 0
    while True:
        i += d_i
        j = (j + d_j) % cols
        if i >= rows:
            break
        if map_t[i][j] == '#':
            trees += 1
    return trees

trees_31 = count_trees_on_slope(map_t, (3,1))
print(f"Trees count: {trees_31}")

slopes = [(1, 1), (3, 1), (5, 1), (7, 1), (1, 2)]
trees_product = 1
for slope in slopes:
    trees_product *= count_trees_on_slope(map_t, slope)
print(f"Trees product for different slopes: {trees_product}")

# Day 4

In [ ]:
passports = []
for pair_list in read_input(4, map_f = lambda x: re.split('\n| ', x), newline='\n\n'):
    pp = {}
    for pair in pair_list:
        k, v = pair.split(':')
        pp[k] = v
    passports.append(pp)

def check_height(s):
    if match:=re.match(r"^([0-9]+)cm$", s):
        if 150 <= int(match.groups()[0]) <= 193:
            return True
        else:
            return False
    elif match:=re.match(r"^([0-9]+)in$", s):
        if 59 <= int(match.groups()[0]) <= 76:
            return True
        else:
            return False
    else:
        return False
        
keys_and_checks = [
    ('byr', lambda x: re.match(r"^[0-9]{4}$", x) and 1920 <= int(x) <= 2002), 
    ('iyr', lambda x: re.match(r"^[0-9]{4}$", x) and 2010 <= int(x) <= 2020), 
    ('eyr', lambda x: re.match(r"^[0-9]{4}$", x) and 2020 <= int(x) <= 2030),
    ('hgt', check_height),
    ('hcl', lambda x: re.match(r"^\#[0-9a-f]{6}$", x)), 
    ('ecl', lambda x: any([x == s for s in ["amb", "blu", "brn", "gry", "grn", "hzl", "oth"]])), 
    ('pid', lambda x: re.match(r"^[0-9]{9}$", x))
]

def count_valid_passports(passports, necessary_keys):
    valid_m1 = 0
    valid_m2 = 0
    for passport in passports:
        if not all([key in passport for key, _ in necessary_keys]):
            continue
        valid_m1 += 1
        if all([f(passport[key]) for key, f in necessary_keys]):
            valid_m2 += 1
    return valid_m1, valid_m2

valid_m1, valid_m2 = count_valid_passports(passports, keys_and_checks)
print(f"Valid passports: {valid_m1}, {valid_m2}")

# Day 5

In [ ]:
rows = 128
cols = 8

def decode_row(s):
    assert(len(s) == 7)
    s = s.replace('B', '1')
    s = s.replace('F', '0')
    return int(s, 2)

def decode_col(s):
    assert(len(s) == 3)
    s = s.replace('L', '0')
    s = s.replace('R', '1')
    return int(s, 2)

def decode_seat(s):
    row = decode_row(s[:7])
    col = decode_col(s[7:])
    return (row, col)

def seat2id(x):
    return x[0] * cols + x[1]

def id2seat(seat_id):
    return (seat_id // cols, seat_id % cols)

seats = set(read_input(5, map_f=decode_seat))

max_id_seat = max(seats, key=seat2id)
print(f"Max seat id: {seat2id(max_id_seat)}")

for i in range(1, rows - 1):
    for j in range(cols):
        if (i, j) not in seats:
            seat_id = seat2id((i, j))
            seat_p = id2seat(seat_id + 1)
            seat_m = id2seat(seat_id - 1)
            if seat_p in seats and seat_m in seats:
                print(f"My seat id: {seat_id}")

# Day 6    

In [ ]:
group_answers = read_input(6, newline='\n\n', map_f=lambda x: re.split(r"\n", x))

def count_answers_anyone(group_answers):
    count = 0
    for group in group_answers:
        count += len(set("".join(group)))
    return count

def count_answers_everyone(group_answers):
    count = 0
    for group in group_answers:
        group_counter = Counter("".join(group))
        for k, v in group_counter.items():
            if v == len(group):
                count += 1
    return count

print(f"Count of questions anyone in group answered: {count_answers_anyone(group_answers)}")
print(f"Count of questions everyone in group answered: {count_answers_everyone(group_answers)}")

# Day 7

In [ ]:
rules_list = read_input(7, map_f=lambda x: re.match(r"([a-z ]+) bags contain ([0-9a-z, ]*)", x).groups())

color2content = defaultdict(list)
color2parents = defaultdict(list)

for bag, content in rules_list:
    bag = bag.strip()
    for count, color in re.findall(r"([0-9]+)([a-z ]+) bag", content):
        count = int(count)
        color = color.strip()
        color2content[bag] += [(count, color)]
        color2parents[color] += [bag]

def count_bags_containing_color(color, color2parents):
    visited = set()
    opened_list = list(color2parents[color])
    while opened_list != []:
        current = opened_list.pop()
        if current in visited:
            continue
        visited.add(current)
        opened_list += color2parents[current]
    return len(visited)

def count_total_bags_contained(color, color2content):
    total = 0
    for count, c in color2content[color]:
        total += count + count * count_total_bags_contained(c, color2content)
    return total

print(f"Total number of bags that can eventually contain shiny gold bag: {count_bags_containing_color('shiny gold', color2parents)}")
print(f"Total number of bags shiny gold bag contains: {count_total_bags_contained('shiny gold', color2content)}")


# Day 8

In [ ]:
instructions = read_input(8, map_f=lambda x: x.split())

def acc_before_instr_repeats(instructions):
    acc = 0
    pc = 0
    executed = set()
    while True:
        if pc in executed:
            return acc, -1
        if pc >= len(instructions):
            return acc, +1
        op, arg = instructions[pc]
        executed.add(pc)
        if op == "acc":
            acc += int(arg)
            pc += 1
        elif op == "jmp":
            pc += int(arg)
        else:
            pc += 1

def fix_code(instructions):
    for i, (op, arg) in enumerate(instructions):
        if op == "jmp":
            instructions[i] = ("nop", arg)
            acc, ret = acc_before_instr_repeats(instructions)
            if ret == 1:
                return acc
            instructions[i] = (op, arg)
        elif op == "nop":
            instructions[i] = ("jmp", arg)
            acc, ret = acc_before_instr_repeats(instructions)
            if ret == 1:
                return acc
            instructions[i] = (op, arg)

print(f"Acc before instruction repeats: {acc_before_instr_repeats(instructions)[0]}")
print(f"Acc after fixing the code: {fix_code(instructions)}")

# Day 9

In [ ]:
numbers = read_input(9, map_f=int)

def sum_exists(num_list, target):
    num_count = Counter(num_list)
    for n1 in num_list:
        n2 = target - n1
        if n1 == n2 and num_count[n1] >= 2:
            return True
        elif n2 in num_count:
            return True
    return False

def find_first_invalid_num(num_list, preamble_len):
    for i, n in zip(range(preamble_len, len(numbers)), numbers[preamble_len:]):
        if not sum_exists(numbers[i-preamble_len: i], n):
            return n

def find_contigous_sum(num_list, target):
    l = 0
    r = 1
    sum_val = num_list[l]
    while True:
        if sum_val == invalid_num:
            return l, r, min(numbers[l:r]) + max(numbers[l:r])
        elif sum_val < invalid_num:
            r += 1
            sum_val += num_list[r - 1]
        elif sum_val > invalid_num:
            sum_val -= num_list[l]
            l += 1

invalid_num = find_first_invalid_num(numbers, 25)
print(f"First invalid number: {invalid_num}")
left, right, min_max_sum = find_contigous_sum(numbers, invalid_num)
print(f"Min max sum: {min_max_sum}")

# Day 10

In [ ]:
adapters = sorted(read_input(10, int))

def count_jolt_differences(adapters):
    diff = Counter(np.array(adapters + [adapters[-1] + 3]) - np.array([0] + adapters))
    return diff[1] * diff[3]

def count_diff_arangements(adapters):
    counts = np.zeros(adapters[-1] + 1)
    counts[0] = 1
    for adapter in adapters:
        counts[adapter] = np.sum(counts[max(0, adapter-3):adapter])
    return counts[-1]

print(f"Multiplied count of differences equal to 3 and 1: {count_jolt_differences(adapters)}")
print(f"Number of different arrangements: {count_diff_arangements(adapters)}")

# Day 11

In [ ]:
initial_map = np.array(read_input(11, map_f = list))

DIRECTIONS = list(itertools.product(range(-1, 2), range(-1, 2)))
DIRECTIONS.remove((0, 0))

def count_occupied_at_equilibrium(current, neighbor_f, occupied_tresh):
    rows, cols = current.shape
    while True:
        new = current.copy()
        for i in range(rows):
            for j in range(cols):
                if current[i, j] == '.':
                    continue
                neighborhood = neighbor_f(current, i, j)
                occupied = np.count_nonzero(neighborhood == '#')
                if (current[i, j] == 'L') and (occupied == 0):
                    new[i, j] = '#'
                if (current[i, j] == '#') and (occupied >= occupied_tresh):
                    new[i, j] = 'L'
        if(np.all(current == new)):
            return np.count_nonzero(current == '#')
        current = new

def find_neighbors1(mat, i, j):
    rows, cols = mat.shape
    neighborhood = {}
    for d_i, d_j in DIRECTIONS:
        n_i, n_j = i + d_i, j + d_j
        if n_i >= rows or n_i < 0 or n_j >= cols or n_j < 0:
            neighborhood[(d_i, d_j)] = '.'
        else:
            neighborhood[(d_i, d_j)] = mat[n_i, n_j]
    return np.array(list(neighborhood.values()))

def find_neighbors2(mat, i, j):
    rows, cols = mat.shape
    neighborhood = {}
    for k in range(1, max(rows, cols)):
        if len(neighborhood.keys()) == len(DIRECTIONS):
            break
        for d_i, d_j in DIRECTIONS:
            if (d_i, d_j) in neighborhood: 
                continue
            n_i, n_j = i + k * d_i, j + k * d_j
            if n_i >= rows or n_i < 0 or n_j >= cols or n_j < 0:
                neighborhood[(d_i, d_j)] = '.'
            elif mat[n_i, n_j] == 'L' or mat[n_i, n_j] == '#':
                neighborhood[(d_i, d_j)] = mat[n_i, n_j]
    
    neighbors = list(neighborhood.values())
    assert len(neighbors) == 8
    return np.array(list(neighborhood.values()))

print("Occupied space at equilibrium first part: ", count_occupied_at_equilibrium(initial_map.copy(), find_neighbors1, 4))
print("Occupied space at equilibrium second part: ", count_occupied_at_equilibrium(initial_map.copy(), find_neighbors2, 5))